In [ ]:
import tensorflow as tf


In [ ]:
# from tf.keras.preprocessing.image import ImageDataGenerator
# from tf.keras.layers.normalization import BatchNormalization
# from tf.keras.models import Sequential
# from tf.keras.layers import Convolution2D, MaxPooling2D
# from keras.layers import Activation, Dropout, Flatten, Dense
# from tf.keras.optimizers import RMSprop


# dimensions of our images.
img_width, img_height = 1920, 1080

In [ ]:
# train_data_dir = '/Users/anchitsaxena/Documents/bikroy_may_logs
# /bikroy_may_images_used/train'
# validation_data_dir = '/Users/anchitsaxena/Documents/bikroy_may_logs
# /bikroy_may_images_used/validation'
# train_data_dir = '/Users/anchitsaxena/Documents/ikman_images
# /watermark_images_19_jan/train'
# validation_data_dir = '/Users/anchitsaxena/Documents/ikman_images
# /watermark_images_19_jan/validation'

train_data_dir = '/Users/dhruvi/saltside/saltside-assets/car-image-datasets-india/train'
validation_data_dir = '/Users/dhruvi/saltside/saltside-assets/car-image-datasets-india/test'

nb_train_samples = 3000
nb_validation_samples = 400
# fro latest bikroy_images_used rl = 0.002 , n = 19 ~80% , n = 47,45 higest ~
#  82%
nb_epoch = 5


In [ ]:




model = tf.keras.models.Sequential()
# resizing
IMG_SIZE = 200
model.add(tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE))

# previous model 24 march
# model.add(Convolution2D(32, 3, 3, input_shape=(3, img_width, img_height)))
# new model 24 march
model.add(tf.keras.layers.Convolution2D(32, 3, 3, input_shape = (3, img_width, img_height)))
# sept 2016 exp changes
# model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
# model.add(Dropout(0.5))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

# march 24
model.add(tf.keras.layers.Convolution2D(32, 3, 3))
# model.add(Convolution2D(64, 3, 3))
# model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
# model.add(Dropout(0.5))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

# jan 19
# model.add(Convolution2D(64, 3, 3))
model.add(tf.keras.layers.Convolution2D(64, 3, 3))
# model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
# model.add(Dropout(0.5))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))

model.add(tf.keras.layers.Flatten())
# fror usual 09 jan
# model.add(Dense(64))
model.add(tf.keras.layers.Dense(64))
# for good training but unstable validate points
# ikman iamge_used epochs = 50 accuracy ~79% n_epo = 62 or 64,65 | 46,52 , 71
#  highest
# model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

# model 19jan
# model.add(Dropout(0.5))
# model.add(Dense(1))

model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('sigmoid'))

In [ ]:


# for changes from 02_sept2016
# fro bikroy accuracy touched to 75 for only 7-8 epochs
# for ikman bluemark 19 jan
rms_prop = tf.keras.optimizers.RMSprop(lr = 0.001, rho = 0.9, epsilon = 1e-08)
# rms_prop = RMSprop(lr=0.002, rho=0.9, epsilon=1e-08)
# sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
# adam = Adam(lr=0.2, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
# use 'adam' as well to see

# INITIAL SETTINGS
model.compile(loss = 'binary_crossentropy',
              optimizer = rms_prop,
              metrics = ['accuracy'])
# CUSTOMIZED SETTINGS
# opt = SGD(lr=0.01)
# model.compile(loss='binary_crossentropy',
#             optimizer=sgd,
#            metrics=['accuracy'])
# this is the augmentation configuration we will use
# fbikroy_may_logs/bikroy_may_imagesor training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = 50,
    class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = 50,
    class_mode = 'binary')
model.fit_generator(train_generator, steps_per_epoch = nb_train_samples,
                    epochs = nb_epoch,
                    validation_data = validation_generator,
                    validation_steps = nb_validation_samples)



In [ ]:


model_json = model.to_json()
# check its accuracy on Monday in offc
open(
    '/Users/anchitsaxena/Documents/ikman_images'
    '/ikman_cars_vehicles_download_28_march/model_download'
    '/image_architecture_11.json',
    'w').write(model_json)
model.save_weights(
    '/Users/anchitsaxena/Documents/ikman_images'
    '/ikman_cars_vehicles_download_28_march/model_download/images_weights_11'
    '.h5',
    overwrite = True)